In [28]:
from PIL import Image
import os

# Input and output folders
input_folder = './train'
output_folder = './train_32x32'

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

def center_crop_to_square(img):
    width, height = img.size
    min_dim = min(width, height)
    left = (width - min_dim) // 2
    top = (height - min_dim) // 2
    right = left + min_dim
    bottom = top + min_dim
    return img.crop((left, top, right, bottom))

for filename in os.listdir(input_folder):
#for filename in ['img_000015.jpg']:
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
        img_path = os.path.join(input_folder, filename)
        img = Image.open(img_path).convert('RGB')  # Ensure 3-channel RGB

        # Crop to square around center
        cropped = center_crop_to_square(img)

        # Resize to 32x32
        resized = cropped.resize((32, 32), Image.Resampling.LANCZOS)

        # Save to output folder
        resized.save(os.path.join(output_folder, filename))

In [29]:
from PIL import Image, ImageEnhance
import os

input_folder = './train_32x32'
output_folder = './train_32x32_gray_contrast'

os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
#for filename in ['img_000015.jpg']:
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
        img_path = os.path.join(input_folder, filename)
        img = Image.open(img_path).convert('L')  # Convert to grayscale

        # Enhance contrast
        enhancer = ImageEnhance.Contrast(img)
        img_contrast = enhancer.enhance(4.0)  # Contrast factor (1.0 = original, >1 = more contrast)

        # Save processed image
        img_contrast.save(os.path.join(output_folder, filename))

In [30]:
from PIL import Image
import numpy as np
import os
import pandas as pd

input_folder = './train_32x32_gray_contrast'
output_csv = 'flattened_images.csv'

data = []

for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
        img_path = os.path.join(input_folder, filename)
        img = Image.open(img_path).convert('L')  # Convert to grayscale
        img_resized = img.resize((32, 32), Image.Resampling.LANCZOS)
        
        # Flatten image to 1D vector
        arr = np.array(img_resized).flatten()  # shape: (1024,)
        
        # Add image filename at the end
        row = arr.tolist() + [filename]
        data.append(row)

# Create column names: pixel_0 ... pixel_1023, image_path
column_names = [f'pixel_{i}' for i in range(1024)] + ['image_path']

# Create DataFrame and save to CSV
df = pd.DataFrame(data, columns=column_names)
df.to_csv(output_csv, index=False)

print(f"Saved {len(df)} image vectors to {output_csv}")

Saved 5488 image vectors to flattened_images.csv
